In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
import hopsworks

api_key = os.getenv('PeeKR8e6oC5gBvft.VLslYEDkMe9EsDw58YEIUfGcir8Rm8kMxx4zIVfHDLFPCr44Y4cEVpvHcnxnEG03')  
project_name = os.getenv('NYC_Taxi_Data')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)

feature_store = project.get_feature_store()
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency", 
    primary_key=["pickup_location_id","pickup_hour"],
    event_time="pickup_hour"
)

2025-03-03 16:22:30,855 INFO: Initializing external client
2025-03-03 16:22:30,858 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 16:22:33,649 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215669


In [5]:
import logging
# Connect to the feature store
feature_store = project.get_feature_store()

# Retrieve the feature view
try:
    feature_view = feature_store.get_feature_view(
        name="time_series_hourly_feature_view",
        version=1,
    )
    logging.info("Feature view retrieved successfully.")
except Exception as e:
    logging.error(f"Error retrieving feature view: {e}")
    exit()  # Exit if the feature view cannot be retrieved.

2025-03-03 16:22:36,129 INFO: Feature view retrieved successfully.


In [6]:
import logging
import hopsworks
logging.basicConfig(level=logging.INFO)
try:
    ts_data, _ = feature_view.training_data(
        description="Time-series hourly taxi rides",
        write_options={"use_spark": True}
    )
    logging.info("Training data created successfully.")
except Exception as e:
    logging.error(f"Error creating training data: {e}")

2025-03-03 16:23:32,250 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 209, in __init__
    self._health_check()
  File "C:\Users\Admin\anaconda3\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\retrying.py", line 266, in call
    raise attempt.get()
          ^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\retrying.py", line 301, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "C:\Users\Admin\anaconda3\Lib\site-packages\six.py", line 719, in reraise
    raise value
  File "C:\Users\Admin\anaconda3\Lib\site-packages\retrying.py", line 251, in call
    attempt = Attempt(fn(*args, **kwargs), attempt_number, False)
                   

2025-03-03 16:23:39,589 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 394, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\hsfs\core\arrow_flight_client.py", line 459, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\retrying.py", line 301, in get
    six.reraise(self.value[0], self.value[1]

In [7]:
ts_data = ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

NameError: name 'ts_data' is not defined

In [ ]:
ts_data.head()

In [ ]:
ts_data["pickup_hour"].min()

In [ ]:
ts_data["pickup_hour"].max()

In [ ]:
ts_data_copy = ts_data.copy()

In [ ]:
import pandas as pd
ts_data["pickup_hour"] = pd.to_datetime(ts_data["pickup_hour"], errors="coerce")

In [ ]:
ts_data.info()

In [ ]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)  # Remove timezone

In [ ]:
ts_data.info()

In [ ]:
ts_data["year_month"] = ts_data["pickup_hour"].dt.to_period("M")  # Year-Month format

In [ ]:
ts_data

In [ ]:
import matplotlib.pyplot as plt

# Group by year_month and count
hour_counts = ts_data.groupby("year_month").size()

# Plot the data
ax = hour_counts.plot(kind="bar", figsize=(10, 6), color="skyblue", edgecolor="black")  # Use 'ax' to store the plot object
ax.set_title("Number of Hours by Year/Month", fontsize=16)
ax.set_xlabel("Year-Month", fontsize=12)
ax.set_ylabel("Count of Hours", fontsize=12)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.grid(axis="y", linestyle="--", alpha=0.7)

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

In [ ]:
ts_data.head()      

In [ ]:
gte = ts_data["year_month"] >= pd.Period("2023-01", freq="M")
lte = ts_data["year_month"] <= pd.Period("2023-12", freq="M")
cond = gte & lte
filtered_data = ts_data[cond].reset_index(drop=True)

In [ ]:
filtered_data.drop(columns=["year_month"], inplace=True) 

In [ ]:
filtered_data.shape

In [ ]:
ts_data = filtered_data

In [ ]:
ts_data.info()

In [ ]:
ts_data.head()["pickup_hour"].values

In [ ]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [ ]:
features.sort_values(["pickup_location_id", "pickup_hour"])

In [ ]:
features[features["pickup_location_id"] == 2]

In [ ]:
features_copy = features.copy()

In [ ]:
features_targets = features.copy()
features_targets["target"] = targets

features_targets.shape

In [ ]:
from datetime import datetime, timedelta  
import pandas as pd  
from src.data_utils import split_time_series_data  

# Define the cutoff date as 28 days before today  
cutoff_date = pd.Timestamp(datetime.now() - timedelta(days=28))  
# # Convert the cutoff date to a string in a specific format (e.g., 'YYYY-MM-DD')  
# cutoff_date_str = cutoff_date.strftime('%Y-%m-%d')
# # Print the string representation of the cutoff date  
# print(f"Cutoff date as string: {cutoff_date_str}")
cutoff_date

In [ ]:
X_train, y_train, X_test, y_test = split_time_series_data(
    features_targets,
    cutoff_date=cutoff_date,
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_train.info()

In [ ]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()

In [ ]:
pipeline.fit(features, targets)

In [ ]:
from sklearn.metrics import mean_absolute_error
predictions = pipeline.predict(features)

In [ ]:
predictions

In [ ]:
test_mae = mean_absolute_error(targets, predictions)
print(f"{test_mae:.4f}")

In [ ]:
import joblib  


# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(features)
output_schema = Schema(targets)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [ ]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor",
    input_example=features.sample(),
    model_schema=model_schema,
    include_files=["src/", "requirements.txt"]  # Include directories and files
)
# https://community.hopsworks.ai/t/attributeerror-windowspath-object-has-no-attribute-startswith/1003
# model.save(config.MODELS_DIR / 'lgb_model.pkl')
model.save('C:\\Users\\Admin\\my-project\\models\\lgb_model.pkl')

In [ ]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

In [ ]:
from src.inference import get_model_predictions

In [ ]:
preds = get_model_predictions(model, features_copy)

In [ ]:
model.get_m

In [ ]:
preds

In [ ]:
test_mae = mean_absolute_error(targets, preds["predicted_demand"])
print(f"{test_mae:.4f}")